### INSERT DATA TO SQL

In [25]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

In [ ]:
# conectar con mysql -- cambiar PASSWORD! ;)
cnx = mysql.connector.connect(
            user='root',
            password='---------',
            host='127.0.0.1')

cursor = cnx.cursor()

In [27]:
cursor.execute("USE musicstream")

In [28]:
cnx.database

'musicstream'

In [29]:
# importar .csv para tener un DataFrame
df_lastfm = pd.read_csv('Artistas_LastFM.csv')
df_spotify = pd.read_csv('Canciones_Spotify.csv')

In [30]:
# error en la linea 72 - año solo 2015

df_spotify.head(73)

,id,track_name,artist_name,year,genre,album_type,release_date,popularity
0,1WHNqqRWhJVZIdCScFKtl5,Washington on Your Side,Leslie Odom Jr.,2015,soundtrack,album,2015-09-25,66
1,1DLfR4MOfLYbV6v3xrmWa8,We Know,Lin-Manuel Miranda,2015,soundtrack,album,2015-09-25,66
2,2AtC6i0b8TjpjhWBZYLprX,Bonetrousle,Toby Fox,2015,soundtrack,album,2015-09-15,58
3,6oF8ueLn5hIl4PRp17sxW6,That Would Be Enough,Phillipa Soo,2015,soundtrack,album,2015-09-25,67
4,46YSff2Rq1ZtN1YVk5cwbZ,Brave Shine,Aimer,2015,soundtrack,album,2015-07-29,61
...,...,...,...,...,...,...,...,...
68,5L9MJsGqzTRD09rSzHkCDy,Home,Toby Fox,2015,soundtrack,album,2015-09-15,62
69,6YTc18izaUvCS2XD7rK9Am,Dance of Dragons,Ramin Djawadi,2015,soundtrack,album,2015-06-09,46
70,3TfKt8mPpdXfQTMfRjHzyz,Cabinet Battle #1,Christopher Jackson,2015,soundtrack,album,2015-09-25,66
71,4EnswQiF7J5MsB18ElZSS2,Esperanza Madre del Amor,Agrupacion Musical Nuestro Padre Jesús Despoja...,2015,soundtrack,album,2015,25


In [31]:
df_lastfm.tail(10)

,artist_name,biography,listeners,playcount,similar_artist
1188,Praiz,"Praise Ugbede Adejo (born 8 March 1984), bette...",20002,118125,illBliss
1189,Irina Barros,https://www.last.fm/music/Irina+Barros,4813,44915,Rui Orlando
1190,PAAX (Tulum),https://www.last.fm/music/PAAX+(Tulum),40020,273449,Antaares
1191,Matt Sawyer,https://www.last.fm/music/Matt+Sawyer,1763,4411,Stereo Kulisse
1192,Cubita,https://www.last.fm/music/Cubita,5400,50332,Nuno Ribeiro
1193,Amaarae,"Ama Serwah Genfi (born July 4, 1994), known pr...",938668,30555467,Rochelle Jordan
1194,Chris Brown,"Christopher Maurice Brown (born May 5, 1989) i...",4606712,147516074,Chris Brown & Tyga
1195,Lola Jane,https://www.last.fm/music/Lola+Jane,5747,50806,Archie & Sizzle
1196,Duda,There are two groups using the name Duda one f...,6673,70358,Peled
1197,Simone Vitullo,Simone Vitullo DJ and house music producer for...,21762,102322,Chambord


In [32]:
#NULOS - convertir NULOS de NaN a None (para que SQL acepte los datos)
df_lastfm  = df_lastfm.where(pd.notnull(df_lastfm), None)
df_spotify = df_spotify.where(pd.notnull(df_spotify), None)

In [37]:
# MySQL - inserir data en la tabla LastFM

sql_lastfm = """INSERT INTO lastfm (artist_name, biography, listeners, playcount, similar_artists)
                VALUES (%s, %s, %s, %s, %s)
"""
cursor.executemany(sql_lastfm, df_lastfm.values.tolist())

IntegrityError: 1062 (23000): Duplicate entry 'Leslie Odom Jr.' for key 'lastfm.PRIMARY'

**Cambiar 'release_date' de   año '2015'   para   año completo '2015-01-01**

In [34]:
# 1. Asegurar que la columna 'release_date' sea de tipo string para operar sobre el formato.
df_spotify['release_date'] = df_spotify['release_date'].astype(str)

# 2. Crear una condición: ¿Tiene el valor exactamente 4 caracteres (solo año)?
# Esto identifica '2015' pero excluye '2015-10-25' (que tiene 10 caracteres).
condicion_solo_anio = (df_spotify['release_date'].str.len() == 4)

# 3. Aplicar la corrección usando .loc:
# En las filas donde la condición es VERDADERA, añadimos '-01-01' al final.
df_spotify.loc[condicion_solo_anio, 'release_date'] = \
    df_spotify.loc[condicion_solo_anio, 'release_date'] + '-01-01'

# 4. (Opcional pero muy recomendado) Convertir la columna al tipo datetime de Pandas
# Esto valida que todos los valores ahora son fechas válidas.
df_spotify['release_date'] = pd.to_datetime(df_spotify['release_date'], errors='coerce')


In [35]:
# revisando el error de la linea 72

df_spotify.head(1407)

,id,track_name,artist_name,year,genre,album_type,release_date,popularity
0,1WHNqqRWhJVZIdCScFKtl5,Washington on Your Side,Leslie Odom Jr.,2015,soundtrack,album,2015-09-25,66
1,1DLfR4MOfLYbV6v3xrmWa8,We Know,Lin-Manuel Miranda,2015,soundtrack,album,2015-09-25,66
2,2AtC6i0b8TjpjhWBZYLprX,Bonetrousle,Toby Fox,2015,soundtrack,album,2015-09-15,58
3,6oF8ueLn5hIl4PRp17sxW6,That Would Be Enough,Phillipa Soo,2015,soundtrack,album,2015-09-25,67
4,46YSff2Rq1ZtN1YVk5cwbZ,Brave Shine,Aimer,2015,soundtrack,album,2015-07-29,61
...,...,...,...,...,...,...,...,...
1402,4YRZwIAblxzxkf78IbUNhp,Gundeme,Hozan Muzaffer,2019,k-pop,album,2019-04-01,0
1403,7djeEwxzTz9ToI03p6RRjM,Neem Ke Patiya Pe,Rajiv Raj,2019,k-pop,compilation,2019-09-25,0
1404,6Dr2LqP1dadmZW3eQLGKim,Do Dat Tih$,Rabbit,2019,k-pop,single,2019-12-29,0
1405,5epJh0RcVEMC4qxsJIikbZ,Telmın Çiye,Hozan Serwan,2019,k-pop,album,2019-06-26,0


In [36]:
# MySQL - inserir data en la tabla Spotify

sql_spotify = """INSERT INTO spotify (id, track_name, artist_name, `year`, genre, album_type, release_date, popularity)
                #VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(sql_spotify, df_spotify.values.tolist())

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '' at line 2

In [13]:
# 1. Identificar Duplicados
# Usamos el campo 'id' (tu clave primaria) para buscar filas duplicadas.
# 'keep=False' marca *todas* las instancias de un duplicado (la primera y las posteriores).
df_duplicados = df_spotify[df_spotify.duplicated(subset=['id'], keep=False)]

# 2. Obtener Solo Filas Únicas
# 'keep="first"' mantiene la primera aparición de cada 'id' único y descarta el resto.
df_unicos = df_spotify.drop_duplicates(subset=['id'], keep='first')

# 3. Mostrar las IDs que serán descartadas
print("--- 🛑 ATENCIÓN: IDs de canciones repetidas descartadas ---")
# Filtramos df_duplicados para mostrar solo las que NO están en df_unicos
ids_duplicadas = df_duplicados[~df_duplicados.index.isin(df_unicos.index)]['id'].unique()

if ids_duplicadas.size == 0:
    print("No se encontraron IDs duplicadas en el DataFrame.")
else:
    print(f"Se encontraron y descartaron {len(ids_duplicadas)} IDs duplicadas:")
    for track_id in ids_duplicadas:
        # Aquí puedes ajustar qué información quieres mostrar (ej: solo la ID)
        print(f"❌ ID: {track_id}")

print("----------------------------------------------------------")
print(f"Datos a insertar: {len(df_unicos)} filas únicas.")


# 4. Insertar los Datos Únicos en MySQL
sql_spotify = """INSERT INTO spotify (id, track_name, artist_name, `year`, genre, album_type, release_date, popularity)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

try:
    # Usamos df_unicos (sin duplicados) para la inserción
    cursor.executemany(sql_spotify, df_unicos.values.tolist())
    # Confirma la transacción en la base de datos
    cnx.commit() 
    print(f"✅ ÉXITO: Se insertaron {cursor.rowcount} filas únicas en la tabla 'spotify'.")
except Exception as e:
    # En caso de que ocurra otro error durante la inserción
    print(f"❌ ERROR durante la inserción final: {e}")
    # Deshace cualquier cambio si la inserción falla
    cnx.rollback()

--- 🛑 ATENCIÓN: IDs de canciones repetidas descartadas ---
Se encontraron y descartaron 61 IDs duplicadas:
❌ ID: 1Ogp6LEB5Qw1kl0pmpRJwo
❌ ID: 5QhyZb93AHFlpznADf9Jsu
❌ ID: 7xvqadKFlrEn3WBdx60FP5
❌ ID: 3gkaxVLKhjpaLiUzGjuSbH
❌ ID: 1jfsYYqVRVdyrbfqwOGdpq
❌ ID: 3GJGMPUXISaQdAAOr7OAm9
❌ ID: 5f4B9bJra0XLDcDRkcYsEK
❌ ID: 2f67tCczV1DxwjsiKhm3wz
❌ ID: 525eXAknaCIBpoxHGHrjwt
❌ ID: 14OWTW3gB3UPDhz0J1lcak
❌ ID: 5oa1hcdOT426gWWZ6ZKPru
❌ ID: 3vVWk5It0E9siRKKn0RInw
❌ ID: 4isDdT0nFh0VXhCTVEEISD
❌ ID: 4qkVb2zemrcfAiHLH2Jc7L
❌ ID: 6g4y38jrL9AqPW7dXVSe8p
❌ ID: 3eLrmO04uxtZPgjL846qbb
❌ ID: 1oc22PLiDwmVKudDNGBTUk
❌ ID: 7xqR92Kowv1VIsiUbJ6t3P
❌ ID: 4ZCU5XdO7HWshWRlWKg5Yf
❌ ID: 3HwnmR5Elvm3aHRRYiz4kr
❌ ID: 0Wu2aurcMw6theY4nQgLPp
❌ ID: 2d1CRXLd1DJHeHFQPG0lF7
❌ ID: 1JbTZIZcEZRGM7J2OyF3ZJ
❌ ID: 7q8FFLCmQ8j9EhtoKyqtx2
❌ ID: 6RqpYSSE8g57EJzEl05al7
❌ ID: 7iBEzm2Pt90mjqWTjQBkfG
❌ ID: 72QBBuwggcOGz0lVmajzfp
❌ ID: 1Xjt165Bwl3DJxuvraFWSL
❌ ID: 65ldmXqHh3sbz2Kbrgg81A
❌ ID: 2rIJInYqXxLtBsr4tv4iVI
❌ ID: 0zWFmIgB6s2bpCB8w

In [14]:
cnx.close()